# Evaluation

In [44]:
import pandas as pd
import json
import requests

SELECT_URL = "http://localhost:5002/solr/articles/select"


In [66]:
# METRICS TABLE
# Define custom decorator to automatically calculate metric based on key
metrics = {}
def metric(f): return metrics.setdefault(f.__name__, f)


@metric
def ap(results, relevant):
    """Average Precision"""
    precision_values = [
        len([
            doc
            for doc in results[:idx]
            if doc in relevant
        ]) / idx
        for idx in range(1, len(results))
    ]
    return sum(precision_values)/len(precision_values)


@metric
def p10(results, relevant, n=10):
    """Precision at N"""
    return len([doc for doc in results[:n] if doc in relevant])/n

@metric
def re(results, relevant):
    """Recall"""
    return len([doc for doc in results if doc in relevant])/len(relevant)


def calculate_metric(key, results, relevant):
    return metrics[key](results, relevant)


# Define metrics to be calculated
evaluation_metrics = {
    'ap': 'Average Precision',
    'p10': 'Precision at 10 (P@10)',
    're': 'Recall',
}


def _parse_articles(articles):
    return list(map(lambda x: "{}/{}/{}".format(x["book"], x["key"], x["date"]), articles))

def metrics_table(search_results, qrels_results):
    search_results = _parse_articles(search_results)

    # Calculate all metrics and export results as LaTeX table
    df = pd.DataFrame([['Metric', 'Value']] +
                      [
        [evaluation_metrics[m], calculate_metric(
            m, search_results, qrels_results)]
        for m in evaluation_metrics
    ]
    )

    print(df)


## Search by articles containing "horas extraordinárias"

In [67]:
# you've got to do this manually
# for this example I searched for "horas extraordinárias" in the regular file
# and these IDs are "book/key/date"
qrels = [
    "Código Penal/58/1995-03-15T00:00:00Z",
    "Código Penal/58/2017-11-21T00:00:00Z",
    "Código Penal/58/2007-09-15T00:00:00Z",
]

results = requests.get(
    SELECT_URL,
    params={
        "defType": "edismax",
        "qf": "title_raw^3 title^2 text_raw^1.5 text^1 path^0.5 book^0.25",
        "bq": "state:Consolidado^4",
        "pf": "title_raw^3 title^2 text_raw^1.5 text^1 path^0.5 book^0.25",
        "q": "horas extraordinárias"
    }
).json()['response']['docs']

# average precision takes the order into account
# precision at 10 is the percentage of relevant information in the top10
# recall is the percentage of relevant information in the total of results
metrics_table(results, qrels)


                        0         1
0                  Metric     Value
1       Average Precision  0.600397
2  Precision at 10 (P@10)       0.3
3                  Recall       1.0
